In [ ]:
from networkx.algorithms.bipartite.basic import color
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.animation import FuncAnimation
import networkx as nx
import statsmodels.api as sm
from glob import glob
from GLVsim import GLV
from sklearn.decomposition import PCA

# change global plotting params
import matplotlib as mpl
font = {'family' : 'normal',
        'weight' : 'normal',
        'size'   : 14}
mpl.rc('font', **font)


 # Fitness
 ## Fitness Over Time
 First we just need to make sure that fitness actually went up otherwise
 what are we even doing here?

In [ ]:
df_list = []
for file in glob('../data/complete/fitness*.csv'):
    df_single = pd.read_csv(file)
    times = df_single['time'].values
    df_list.append(df_single.drop('time', axis = 1).values)

fitnesses = np.array(df_list).mean(axis=0)[:, :-1]
avg_fit = fitnesses.mean(axis=1)
max_fit = fitnesses.max(axis=1)
plt.figure(figsize=(5,4))
plt.plot(times, fitnesses, c='grey', alpha=0.2)

plt.plot(times, avg_fit, c='C0', label = 'Average')
plt.plot(times, max_fit, c='C1', label = 'Max')
plt.legend()
plt.xlabel('GA Tournaments')
plt.ylabel('Average Species Richness')
plt.tight_layout()
plt.savefig('../figures/fitness.png')
plt.savefig('../figures/fitness.pdf')
plt.show()


 ## Fitness Residuals
 Hell yeah. We also want to make sure that mean fitness is actually a good description
 of the tendency of our system. Let's double check that

In [ ]:
residuals = (fitnesses.T - avg_fit).flatten()
sns.histplot(residuals)
plt.xlabel('Residual fitness')
plt.savefig('../figures/residual-fitness.png')
plt.savefig('../figures/residual-fitness.pdf')


 It looks pretty good like we have a normal-ish distribution. So we can use
 the mean to characterize the tendancy.

 # Demo Time Series
 We're going to want to put some little example timeseries in the paper. Is my
 library installed? First we'll get the lowest fitness network than the highest
 it might be cool to run each of these like a bunch of times and see where they
 end up in pca space.

In [ ]:

# lowest fitness indices
low_fit = np.where(fitnesses == np.min(fitnesses))
low_time = times[low_fit[0]][0]
low_indiv = low_fit[1][0]

# pull the lowest fitness network
low_adj = np.loadtxt('../data/complete/networks1/{}/{}_adjmat_10.csv'.format(low_time, low_indiv), delimiter=',')
np.fill_diagonal(low_adj, 1)

# highest fitnesses indices
high_fit = np.where(fitnesses == np.max(fitnesses))
high_time = times[high_fit[0]][0]
high_indiv = high_fit[1][0]

# pull the lowest fitness network
high_adj = np.loadtxt('../data/complete/networks1/{}/{}_adjmat_10.csv'.format(high_time, high_indiv), delimiter=',')
np.fill_diagonal(high_adj, 1)

In [ ]:
# params
num_weights = 45
runs = 20
low_vecs = np.zeros((num_weights * runs, 10))
high_vecs = np.zeros((num_weights * runs, 10))
labels = np.hstack((np.repeat([0], runs), np.repeat([1], runs)))
weights = -np.random.exponential(size=(num_weights, 10, 10))
ri = np.random.uniform(size=(runs, 10))

for w in range(num_weights):
    # loop for each set of weights
    for i in range(runs):
        # low sim
        glv.set_state(ri[i])
        glv.set_matrix(low_adj * weights[w])
        _, ls = glv.simulate(50, 0.01)
        low_vecs[runs * w + i] = ls[-1]

        # high vecs
        glv.set_state(ri[i])
        glv.set_matrix(high_adj * weights[w])
        _, hs = glv.simulate(50, 0.01)
        high_vecs[runs * w + i] = hs[-1]



In [ ]:
vecs = np.vstack((low_vecs, high_vecs))
pca = PCA()
vecs_pca = pca.fit_transform(vecs)


# lets make some plots
expvar = pca.explained_variance_ / np.sum(pca.explained_variance_)
ev_cs = np.cumsum(expvar)

fig, ax = plt.subplots(ncols=2, figsize=(10, 5))

# run to highlight
wi = 25
ii = 10

# regular old PCA plot
ax[1].axvline(0, linestyle='--', color='grey', alpha=0.2)
ax[1].axhline(0, linestyle='--', color='grey', alpha=0.2)
ax[1].scatter(vecs_pca[runs * num_weights:, 0], vecs_pca[runs * num_weights:, 1], 
              label='High Fitness', marker='x', alpha=0.6, facecolor='C1', edgecolor='C1')
ax[1].scatter(vecs_pca[:runs * num_weights, 0], vecs_pca[:runs * num_weights, 1], label='Low Fitness', 
              alpha=0.6, facecolor='none', edgecolor='C0')
ax[1].scatter(vecs_pca[runs * num_weights + (wi * runs + ii), 0], 
              vecs_pca[runs * num_weights + (wi * runs + ii), 1], s=500, 
              facecolor='none', edgecolor='black')
ax[1].set_xlabel('PC1 ({:.1f}%)'.format(expvar[0] * 100))
ax[1].set_ylabel('PC2 ({:.1f}%)'.format(expvar[1] * 100))
ax[1].legend()

# explained variance plot
# ax[1].bar(range(expvar.shape[0]), expvar, color='grey')
# ax[1].step(range(expvar.shape[0]), ev_cs, color='k')
# ax[1].set_xticks(range(expvar.shape[0]))
# ax[1].set_xlabel('Component #')
# ax[1].set_ylabel('Variance Explained (%)')

# example time series plot
glv = GLV(10)
glv.set_matrix(high_adj * weights[wi])
glv.set_state(ri[ii])
lt, ls = glv.simulate(40, 0.01)
ax[0].plot(lt, ls)
ax[0].set_xlabel('Time')
ax[0].set_ylabel('Species Abundance')

plt.tight_layout()
plt.savefig('../figures/pca.pdf')
plt.savefig('../figures/pca.png')
plt.show()
